# Práctico: Introducción a Modelos de Lenguaje usando Hugging Face y GPT-2

In [ ]:
# Primero, instala la librería transformers si no está instalada
# Descomenta la línea de abajo si necesitas instalar la librería
# !pip install transformers

In [ ]:
# Importando las librerías necesarias
import time
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import logging as transformers_logging

# Configurar el logging para que no muestre warnings de transformers
transformers_logging.set_verbosity_error()

## 1. Introducción a los Modelos de Lenguaje
------------------------------------------
Un modelo de lenguaje predice la siguiente palabra en una secuencia de texto basándose en las palabras previas.

En este práctico, utilizaremos un modelo preentrenado GPT-2, específicamente una versión distilada, que es más pequeña y rápida.

## 2. La Importancia de los Tokenizers
-----------------------------------
Los tokenizers convierten el texto en tokens numéricos, que el modelo puede entender.
Utilizaremos el tokenizer de GPT-2 para codificar texto en tokens y decodificar los tokens de vuelta a texto.

In [ ]:
# Cargando el tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilgpt2')

In [ ]:
# Ejemplo: Tokenizando una oración
sentence = "A long time ago in a galaxy far, far away there was an astrophysicist"

In [ ]:
# Tokenizar la oración
tokens = tokenizer.encode(sentence)
print("Oración tokenizada:", tokens)

Los tokens son los números que representan las palabras o partes de las palabras.

In [ ]:
# Decodificando los tokens de nuevo a texto
decoded_sentence = tokenizer.decode(tokens)
print("Oración decodificada:", decoded_sentence)

### Tokenizador de GPT-2 vs Tokenizador BERT.

In [ ]:
# Cargar el tokenizer de BERT
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

print("BERT tokenizer:")

# Tokenizar con BERT
bert_tokens = ...
print("- Tokens:", bert_tokens)
print("- Cantidad de tokens:", len(bert_tokens))

# Tokens individuales
indivitual_tokens = bert_tokenizer.convert_ids_to_tokens(bert_tokens)
print("- Tokens individuales: ", indivitual_tokens)

# Decodificar los tokens de BERT
bert_decoded_sentence = ...
print("- Oración decodificada:", bert_decoded_sentence, '\n')


print("GPT-2 tokenizer:")
# Comparar con el tokenizer de GPT-2
gpt2_tokens = ...
print("- Tokens:", gpt2_tokens)
print("- Cantidad de tokens:", len(gpt2_tokens))

# Tokens individuales
gpt2_indivitual_tokens = ...
print("- Tokens individuales: ", gpt2_indivitual_tokens)

# Decodificar los tokens de GPT-2
gpt2_decoded_sentence = ...
print("- Oración decodificada:", gpt2_decoded_sentence)


### Ejercicio:
- ¿Qué son los tokens `[CLS]` y `[SEP]` que aparecen en la *Oracion decodificada* de BERT?
- ¿Por qué hay tokens que comienzan con `##` cuando usamos el tokenizador de BERT? ¿Qué significan? ¿Y los que no lo tienen?
- ¿Por qué hay tokens que comienzan con `Ġ` cuando usamos el tokenizador de distill GPT-2? ¿Qué significan? ¿Y los que no lo tienen?
- ¿Cree que son necesarios estos tokens? ¿Por qué?
- ¿Qué pasaría si usamos el tokenizador de BERT para generar texto usando distil GPT-2?

## 3. Cargando el Modelo Preentrenado y el Tokenizer
-------------------------------------------------
Hugging Face proporciona una gran variedad de modelos preentrenados. Usaremos el modelo 'distilgpt2' para este ejercicio.

In [ ]:
# Cargar el modelo
model = AutoModelForCausalLM.from_pretrained('distilgpt2')

In [ ]:
# Poner el modelo en modo de evaluación
model.eval()

## 4. Salida Cruda (Logits)
--------------------------------------------
Obtendremos la salida cruda del modelo (logits).

Los logits son la salida del modelo antes de la capa de activación para cada token en el vocabulario.

In [ ]:
# Ejemplo de prompt
prompt = "Yesterday, I dreamed of a world where"

In [ ]:
# Tokenizar la entrada
inputs = tokenizer(prompt, return_tensors="pt")

In [ ]:
inputs["input_ids"].shape

In [ ]:
# Pasar la entrada por el modelo para obtener los logits (puntajes sin normalizar)
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits  # Salida cruda del modelo

In [ ]:
logits

In [ ]:
logits.shape

In [ ]:
next_token_logits = logits[:, -1, :]  # Logits para el último token en el prompt

### **Ejercicio:**

- Explique las dimensiones de los logits.
- ¿Qué función de activación usaría?
- Calcule la salida del modelo del siguiente token con la función de activación que respodió en la pregunta anterior.
- ¿Qué token elegiría y por qué?

## 5. Selección Manual de Tokens: Top-K y Top-P Sampling
-----------------------------------------------------

## Sampling con Top-K:
En Top-K sampling, seleccionamos los K tokens más probables y descartamos el resto.

Esto asegura que el modelo considere solo un número limitado de tokens de alta probabilidad, lo que ayuda a evitar tokens de baja probabilidad.

In [ ]:
# Función para aplicar Top-K sampling
def top_k_sampling(logits, k=50):
    # Mantener solo los K tokens con la mayor probabilidad
    # Samplear con los tokens restantes
    # Devolver el token sampleado
    next_token = ...
    return next_token

### Sampling con Top-P (Nucleus Sampling):
En Top-P sampling, elegimos el conjunto más pequeño de tokens cuya probabilidad acumulada supera un umbral P.

Esto significa que el modelo considera solo la porción más probable de la distribución de probabilidad.

In [ ]:
# Función para aplicar Top-P (Nucleus) sampling
def top_p_sampling(logits, p=0.9):
    # Calcular la probabilidad acumulada de los tokens ordenados
    # Remover tokens cuya probabilidad acumulada sobrepasa el umbral p
    # Samplear usando los tokens restantes
    # Mapear el índice de vuelta al espacio original
    next_token = ...
    return next_token


## 6. Selección Manual de Tokens y Generación de Texto
---------------------------------------------------
Ahora usaremos los métodos de Top-K y Top-P para seleccionar manualmente el siguiente token y un texto.

In [ ]:
# Función para generación manual basada en logits
def manual_text_generation(model, tokenizer, prompt, max_length=50, sampling_strategy=top_k_sampling, top_k=50, top_p=0.9):
    """
    1) Codifique el prompt inicial
    2) Configure un loop para generar tokens hasta alcanzar max_length o encontrar un token de fin de secuencia
    3) En cada iteración del loop:
        3.1) Obtenga los logits del modelo para el input
        3.2) Samplear el siguiente token usando la estrategia de muestreo elegida (Top-K o Top-P)
        3.3) Decodificar los tokens generado
        3.4) Añadir el token predicho a la secuencia de entrada y continuar generando texto
    4) Devuelva el texto generado decodificado
    """
    ...


### Ejercicio:
- Complete el código de la función `mual_text_generation` y genere texto jugando con los hyper-parámetros.
- Compare salidas con las estrategias de sampleo `top p` y `top k`.

## 7. Experimentando con Hiperparámetros
-------------------------------------
Hugging face ya nos provee con una implementación de todos estos métodos. Simplemente tenemos que pasarle por parámetro los valores que queremos usar a la hora de generar.

Ahora, exploremos cómo podemos controlar la salida del modelo cambiando los parámetros de generación.

Intenta cambiar los valores de `max_length`, `temperature`, `top_k` y `top_p`.

In [ ]:
# Tokenizar la entrada
inputs = tokenizer(prompt, return_tensors="pt")

In [ ]:
# Experimentar con diferentes configuraciones
output = model.generate(
    **inputs,
    max_length=20,
    num_return_sequences=1,
    temperature=0.7,
    top_k=50,
    top_p=0.9
)

In [ ]:
# Decodificar y mostrar el texto generado
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("\nTexto generado con parámetros modificados:\n", generated_text.replace('\n', ' '))

### Ejercicio:
- Compare este resultado con el generado en el punto anterior.
- ¿Cómo afecta la `temperatura` a los resultados del modelo?
- ¿Cómo afecta el `top_p` a los resultados del modelo?
- ¿Cómo afecta la `top_k` a los resultados del modelo?

## 8. Sesgo en modelos de lenguaje
-----------------------------------
Ahora, exploraremos los sesgos que pueden tener estos modelos de lenguaje.

In [ ]:
# Ejercicio: Detectar sesgos en la generación de texto usando diferentes prompts
prompts = [...]

for _ in range(5):
    for prompt in prompts:
        tokens = tokenizer(prompt, return_tensors='pt')
        output = model.generate(
            **tokens,
            max_length=20,
            temperature=0.7,
            top_k=50,
            top_p=.9,
            do_sample=True,
        )
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        print("Prompt: ",prompt)
        print("Texto generado: ", generated_text.replace('\n', ' '))
        print("-" * 50)


### Ejercicio:
- ¿Nota algún sesgo en las respuestas de este modelo?
- Proponga prompts que puedan detectar sesgos en este modelo.
- ¿A qué se deben estos sesgos?

## 9. Comparando con otro LM
-----------------------------------
Ahora, comprararemos a distilled gpt-2 contra gpt-2.

In [ ]:
# Cargar modelo distilado y completo para comparar
distil_gpt2_model = AutoModelForCausalLM.from_pretrained('distilgpt2')
gpt2_model = AutoModelForCausalLM.from_pretrained('gpt2')  # GPT-2 completo

gpt2_tokenizer = AutoTokenizer.from_pretrained('gpt2')

In [ ]:
distil_gpt2_model.eval()
gpt2_model.eval()

In [ ]:
# Puedes ejecutar el mismo texto para ambos modelos
prompt = "Photosyntheis is a process that"
max_length = 20

# Tokenizamos el texto


print("Texto de entrada:", prompt)

# Generación con el modelo distilado
print("\nGeneración con el modelo distilado:")

inference_time_distil = ...
generated_text_distil = ...

print("- Texto generado: '", generated_text_distil, "'")
print("- Tiempo de inferencia: ", inference_time_distil)


# Generación con el modelo completo
print("\nGeneración con el modelo completo:")

inference_time_gpt2 = ...
generated_text_gpt2 = ...

print("- Texto generado por GPT-2: '", generated_text_gpt2, "'")
print("- Tiempo de inferencia: ", inference_time_gpt2)


### Ejercicio:

- Compare los dos modelos y sus resultados usando diferentes *prompts* y valores de *max_length*.
- ¿Es necesario usar el parámetro `max_legth`? ¿Por qué?

## 10. Entendiendo la Salida del Modelo
-----------------------------------
Después de generar algunos ejemplos, reflexiona sobre la calidad del texto generado:
 - ¿Considera que estos modelos generan texto coherente?
 - ¿Qué tan creativo o repetitivo es el texto?
 - ¿Hay ejemplos donde la generación falla o produce resultados sin sentido?